In [9]:
# 챗봇, 번역, 이미지 캡셔닝등에 사용되는 시퀀스 학습/생성 모델인 Seq2Seq 을 구현해봅니다.
# 영어 단어를 한국어 단어로 번역하는 프로그램을 만들어봅니다.
import tensorflow as tf
import numpy as np



char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = [['word', '단어'], ['wood', '나무'],
            ['game', '놀이'], ['girl', '소녀'],
            ['kiss', '키스'], ['love', '사랑']]




def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:
        # 인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 만든다.
        input = [num_dic[n] for n in seq[0]]
        # 디코더 셀의 입력값. 시작을 나타내는 S 심볼을 맨 앞에 붙여준다.
        output = [num_dic[n] for n in ('S' + seq[1])]
        # 학습을 위해 비교할 디코더 셀의 출력값. 끝나는 것을 알려주기 위해 마지막에 E 를 붙인다.
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        # 출력값만 one-hot 인코딩이 아님 (sparse_softmax_cross_entropy_with_logits 사용)
        target_batch.append(target)

    return input_batch, output_batch, target_batch


with tf.Graph().as_default():
    #########
    # 옵션 설정
    ######
    learning_rate = 0.01
    n_hidden = 128
    total_epoch = 100
    # 입력과 출력의 형태가 one-hot 인코딩으로 같으므로 크기도 같다.
    n_class = n_input = dic_len


    #########
    # 신경망 모델 구성
    ######
    # Seq2Seq 모델은 인코더의 입력과 디코더의 입력의 형식이 같다.
    # [batch size, time steps, input size]
    enc_input = tf.placeholder(tf.float32, [None, None, n_input])
    dec_input = tf.placeholder(tf.float32, [None, None, n_input])
    # [batch size, time steps]
    targets = tf.placeholder(tf.int64, [None, None])

    
    

    with tf.variable_scope('encode'):
        enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
        outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,dtype=tf.float32)
    with tf.variable_scope('decode'):
        dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
        outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,initial_state=enc_states,dtype=tf.float32)
    model = tf.layers.dense(outputs, n_class, activation=None)
    cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


    
    
    
    #########
    # 신경망 모델 학습
    ######
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    input_batch, output_batch, target_batch = make_batch(seq_data)

    for epoch in range(total_epoch):
        _, loss = sess.run([optimizer, cost],
                           feed_dict={enc_input: input_batch,
                                      dec_input: output_batch,
                                      targets: target_batch})

        print('Epoch:', '%04d' % (epoch + 1),
              'cost =', '{:.6f}'.format(loss))

    print('최적화 완료!')


#########
# 번역 테스트
######
# 단어를 입력받아 번역 단어를 예측하고 디코딩하는 함수
def translate(word):
    # 이 모델은 입력값과 출력값 데이터로 [영어단어, 한글단어] 사용하지만,
    # 예측시에는 한글단어를 알지 못하므로, 디코더의 입출력값을 의미 없는 값인 P 값으로 채운다.
    # ['word', 'PPPP']
    seq_data = [word, 'P' * len(word)]

    input_batch, output_batch, target_batch = make_batch([seq_data])

    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
    prediction = tf.argmax(model, 2)

    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})

    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다.
    decoded = [char_arr[i] for i in result[0]]
    print(decoded)
    # 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로 만든다.
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    return translated


print('\n=== 번역 테스트 ===')

print('word ->', translate('word'))
print('wood ->', translate('wood'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))


Epoch: 0001 cost = 3.741510
Epoch: 0002 cost = 2.578790
Epoch: 0003 cost = 1.693112
Epoch: 0004 cost = 1.190939
Epoch: 0005 cost = 0.775840
Epoch: 0006 cost = 0.545061
Epoch: 0007 cost = 0.444748
Epoch: 0008 cost = 0.217177
Epoch: 0009 cost = 0.255818
Epoch: 0010 cost = 0.095202
Epoch: 0011 cost = 0.224953
Epoch: 0012 cost = 0.102686
Epoch: 0013 cost = 0.265513
Epoch: 0014 cost = 0.153850
Epoch: 0015 cost = 0.225920
Epoch: 0016 cost = 0.126702
Epoch: 0017 cost = 0.106008
Epoch: 0018 cost = 0.089629
Epoch: 0019 cost = 0.118627
Epoch: 0020 cost = 0.173775
Epoch: 0021 cost = 0.122213
Epoch: 0022 cost = 0.018303
Epoch: 0023 cost = 0.088143
Epoch: 0024 cost = 0.038185
Epoch: 0025 cost = 0.079219
Epoch: 0026 cost = 0.016212
Epoch: 0027 cost = 0.012211
Epoch: 0028 cost = 0.010535
Epoch: 0029 cost = 0.012476
Epoch: 0030 cost = 0.009746
Epoch: 0031 cost = 0.006057
Epoch: 0032 cost = 0.008204
Epoch: 0033 cost = 0.005565
Epoch: 0034 cost = 0.009653
Epoch: 0035 cost = 0.002455
Epoch: 0036 cost = 0

In [2]:
num_dic

{'S': 0,
 'E': 1,
 'P': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28,
 '단': 29,
 '어': 30,
 '나': 31,
 '무': 32,
 '놀': 33,
 '이': 34,
 '소': 35,
 '녀': 36,
 '키': 37,
 '스': 38,
 '사': 39,
 '랑': 40}

In [3]:
a = [
    [1,2,3],
    [4,5,6],
    [7,8,9],
    [10,11,12]
]
aa = tf.constant(a)
b = tf.Variable(tf.random_normal(shape=[2,3,4]))
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(b.eval())
    print(tf.argmax(b,0))

[[[-0.2462969  -0.60318005 -0.20894659 -1.4655111 ]
  [-0.9644231  -0.07087076  0.06395355 -0.40863508]
  [ 1.3971809   1.1103567   0.32896623  0.32413498]]

 [[ 0.5102485   0.07214127 -0.8609563  -0.6747008 ]
  [-0.24968834 -0.6733446  -0.9170721   1.6436831 ]
  [ 0.29430234  1.8350353   1.4811567  -0.02238507]]]
Tensor("ArgMax_5:0", shape=(3, 4), dtype=int64)
